In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/Colab\ Notebooks/GCN

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/GCN


In [ ]:
!pip install recommenders[examples]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
def gpu(): # 'warm up' the gpu
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
gpu()
import random
tf.get_logger().setLevel('ERROR') # only display errors

import recommenders
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
# from recommenders.datasets import movielens // does not work?
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from sklearn.model_selection import train_test_split

print(sys.version)
print(pd.__version__)
print(tf.__version__)

3.8.15 (default, Oct 12 2022, 19:14:39) 
[GCC 7.5.0]
1.3.5
2.9.2


In [ ]:
# read movielens .dat file
df_raw = [i.strip().split() for i in open('ml-100k/u.data').readlines()] # 100k data
# df_raw = [i.strip().split('::') for i in open('ml-1m-2/ratings.dat').readlines()] # 1M data
df = pd.DataFrame(df_raw, columns=['userID', 'itemID', 'rating', 'timestamp'])
df['rating'] = df['rating'].astype(int)
train, test = train_test_split(df, test_size=0.15, random_state=14, stratify=df['userID'])

# .yaml file prepared by Miguel Fierro & team, retrieved from GitHub, for LightGCN params
yaml_file = 'lightgcn.yaml'

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

data = ImplicitCF(train=train, test=test, seed=14)
hparams = prepare_hparams(yaml_file, # hparams for 100k data
                          n_layers=3,
                          batch_size=1024,
                          epochs=100,
                          learning_rate=0.0025,
                          decay=0.0001,
                          eval_epoch=1,
                          top_k=20)

# hparams = prepare_hparams(yaml_file, # hparams for 1M data
#                           n_layers=3,
#                           batch_size=1024,
#                           epochs=100,
#                           learning_rate=0.0005,
#                           decay=0.0001,
#                           eval_epoch=1,
#                           top_k=20)

model = LightGCN(hparams, data, seed=14)
with Timer() as train_time: 
  model.fit()


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)1.2s + (eval)0.3s: train loss = 0.55448 = (mf)0.55437 + (embed)0.00011, recall = 0.19267, ndcg = 0.18692, precision = 0.11124, map = 0.07041
Epoch 2 (train)0.9s + (eval)0.2s: train loss = 0.31842 = (mf)0.31795 + (embed)0.00047, recall = 0.20618, ndcg = 0.19672, precision = 0.11569, map = 0.07516
Epoch 3 (train)0.9s + (eval)0.2s: train loss = 0.28779 = (mf)0.28720 + (embed)0.00060, recall = 0.23142, ndcg = 0.22372, precision = 0.13378, map = 0.08767
Epoch 4 (train)0.9s + (eval)0.2s: train loss = 0.26770 = (mf)0.26701 + (embed)0.00068, recall = 0.25130, ndcg = 0.25033, precision = 0.15021, map = 0.10121
Epoch 5 (train)0.9s + (eval)0.2s: train loss = 0.24691 = (mf)0.24613 + (embed)0.00079, recall = 0.25833, ndcg = 0.26377, precision = 0.15753, map = 0.10875
Epoch 6 (train)0.9s + (eval)0.2s: train loss = 0.23665 = (mf)0.23577 + (embed)0.00089, recall = 0.26275, ndcg = 0.27102, p

In [ ]:
print(f"Took {train_time.interval} seconds for training.")

Took 123.6348214589998 seconds for training.


In [ ]:
top_20_all = model.recommend_k_items(test, top_k=20)
top_20_all

,userID,itemID,prediction
0,157,121,9.560719
1,157,257,9.517582
2,157,117,8.643934
3,157,181,8.565247
4,157,222,8.480099
...,...,...,...
18855,400,751,8.040581
18856,400,243,7.927198
18857,400,682,7.901113
18858,400,340,7.857331


In [ ]:
unique_users = top_20_all['userID'].unique()

users_with_hits = 0
hit_indices = []

for uid in unique_users:
    ranked_items = top_20_all.loc[top_20_all['userID']==uid, 'itemID']
    relevant_items = test.loc[test['userID']==uid, 'itemID']
    hits = list(set(ranked_items.values) & set(relevant_items.values))
    if len(hits)>0:
        users_with_hits += 1
    hit_indices.append(ranked_items.where(ranked_items.isin(hits)).dropna().index.values)


In [ ]:
users_with_hits/len(unique_users)

0.9480381760339343

In [ ]:
flat_hit_indices = np.concatenate(hit_indices)
top_20_rmse = top_20_all[top_20_all.index.isin(flat_hit_indices)]

In [ ]:
merged = pd.merge(top_20_rmse, test,  how='left', left_on=['userID','itemID'], right_on = ['userID','itemID'])
merged

,userID,itemID,prediction,rating,timestamp
0,157,117,8.643934,5,886890296
1,157,274,7.629445,4,886890835
2,157,276,7.494263,4,886889876
3,157,127,7.472298,5,886890541
4,482,294,10.694486,4,887643365
...,...,...,...,...,...
4428,356,689,7.529697,5,891406372
4429,356,292,7.454817,3,891405978
4430,317,748,9.351494,5,891446843
4431,317,350,7.591377,5,891446819


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(1,5))
scaler.fit(merged['prediction'].to_numpy().reshape(-1,1))
merged['scaled'] = scaler.transform(merged['prediction'].to_numpy().reshape(-1,1))

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(merged['rating'].to_numpy(), merged['prediction'].to_numpy(), squared=False)

3.7605594918987726

In [ ]:
mean_squared_error(merged['rating'].to_numpy(), merged['scaled'].to_numpy(), squared=False)

1.549137240630569